In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

Step 1: Load images from images folder to pegasus replica catalog <br>
Step 2: Add python script that resizes the images to a transformation catalog <br>
Step 3: Define the outputs of this transformation as pegasus File objects <br>
Step 4: Create/define a job that executes the transformation <br>
Step 5: Define a pegasus workflow <br>

In [2]:
import glob
imagesList= glob.glob('Images/*.jpeg')

In [3]:
imagesList

['Images/img2.jpeg', 'Images/img3.jpeg', 'Images/img1.jpeg', 'Images/img.jpeg']

In [4]:
import logging

from Pegasus.api import *

logging.basicConfig(level=logging.DEBUG)

In [5]:
rc=ReplicaCatalog()
inputFiles = []
for img in imagesList:
    fileName= img.split("/")[1]
    img_file = File(fileName)
    inputFiles.append(img_file)
    rc.add_replica("local", img_file, "/home/scitech/shared-data/sample-images-wf/"+str(img))
rc.write()

In [6]:
inputFiles

[<File img2.jpeg>, <File img3.jpeg>, <File img1.jpeg>, <File img.jpeg>]

In [7]:
image_resize = Transformation(
                "image_resize",
                site="local",
                pfn="/home/scitech/shared-data/sample-images-wf/img_resize.py",
                is_stageable=True
            )

In [8]:
tc = TransformationCatalog()
tc.add_transformations(image_resize)
tc.write()

In [9]:
resizeImages=[]
counter=0
for img in imagesList:
    resize_img = File("image_"+str(counter) +".jpeg")
    resizeImages.append(resize_img)
    counter+=1   

In [10]:
resizeImages

[<File image_0.jpeg>,
 <File image_1.jpeg>,
 <File image_2.jpeg>,
 <File image_3.jpeg>]

In [11]:
wf = Workflow("image_resize_wf")

# resize images
job_image_resize = Job(image_resize)
job_image_resize.add_inputs(*inputFiles)
job_image_resize.add_outputs(*resizeImages)

In [12]:
# add jobs to workflow
wf.add_jobs(
    job_image_resize
)

# run workflow
try:
    wf.plan(submit=True)
    wf.wait()
    wf.statistics()
except PegasusClientError as e:
    print(e.output)


################
# pegasus-plan #
################
[main] WARN  schema.JsonMetaSchema  - Unknown keyword $defs - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword additionalItems - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword examples - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
2020.08.15 01:26:07.153 UTC:
2020.08.15 01:26:07.184 UTC:   -----------------------------------------------------------------------
2020.08.15 01:26:07.202 UTC:   File for submitting this DAG to HTCondor           : image_resize_wf-0.dag.condor.sub
2020.08.15 01:26:07.218 UTC:   Log of DAGMan debugging messages                 : image_resize_wf-0.dag.dagman.out
2020.08.15 01:26:07.243 UTC:   Log

[##################################################] 100.0% ..Success (Completed: 9, Queued: 0, Running: 0, Failed: 0)



######################
# pegasus-statistics #
######################
Your database is compatible with Pegasus version: 5.0.0dev

#
# Pegasus Workflow Management System - http://pegasus.isi.edu
#
# Workflow summary:
#   Summary of the workflow execution. It shows total
#   tasks/jobs/sub workflows run, how many succeeded/failed etc.
#   In case of hierarchical workflow the calculation shows the
#   statistics across all the sub workflows.It shows the following
#   statistics about tasks, jobs and sub workflows.
#     * Succeeded - total count of succeeded tasks/jobs/sub workflows.
#     * Failed - total count of failed tasks/jobs/sub workflows.
#     * Incomplete - total count of tasks/jobs/sub workflows that are
#       not in succeeded or failed state. This includes all the jobs
#       that are not submitted, submitted but not completed etc. This
#       is calculated as  difference between 'total' count and sum of
#       'succeeded' and 'failed' count.
#     * Total - total count 

In [11]:
! pwd

/home/scitech/shared-data/sample-images-wf
